In [ ]:
"""
Created on Mon Feb 05 16:56 2024

Look at historical stuff, based on Davison output

Author: @claraburgard

"""

In [ ]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
import seaborn as sns
import multimelt.useful_functions as uf
import os

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

In [ ]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
outputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_raw = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/'

In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

In [ ]:
inputpath_davison = home_path+'/DATA/SUMMER_PAPER/raw/'
#for vv in ['SMB','calving','melt']:
    
davison_file = xr.open_dataset(inputpath_davison + 'steadystate_davison23.nc')

In [ ]:
def figure_check_hist(isf_list,isf_name,obs,obs_unc,mod,obs2,obs2_unc,obs2_bool=False):
    regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
    colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
              'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

    f = plt.figure()
    f.set_size_inches(8.25*3, 8.25*3)

    ax={}

    leg_hdl = []

    i = 0


    for kisf in tqdm(isf_list):

        ax[i] = f.add_subplot(8,8,i+1)
            
            
        for m,mmod in enumerate(mod.model):
            if 'param' in mod.dims:
                for n,mparam in enumerate(mod.param):

                    ax[i].plot(mod.time, mod.sel(param=mparam,Nisf=kisf,model=mmod), color='grey', alpha=0.2)
            else:
                ax[i].plot(mod.time, mod.sel(Nisf=kisf,model=mmod), color=colors[m])
                
        melt_obs_kisf = obs.sel(Nisf=kisf).values
        melt_obs_unc_kisf = obs_unc.sel(Nisf=kisf).values
        ax[i].axhline(y=melt_obs_kisf,color='k')
        ax[i].fill_between(x=np.arange(1980,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='lightgrey')
        
        if obs2_bool:
            
            melt_obs_kisf = obs2.sel(Nisf=kisf).values
            melt_obs_unc_kisf = obs2_unc.sel(Nisf=kisf).values
            ax[i].axhline(y=melt_obs_kisf,color='k',linestyle='--')
            ax[i].fill_between(x=np.arange(1980,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='orange',alpha=0.3)


        #if kisf == 23:
        #    ax[i].set_title('Tracy Tremenchus')
        #elif kisf == 24:
        #    ax[i].set_title('Conger/Glenzer')
        #elif kisf == 110:
        #    ax[i].set_title('Ekström')
        #else:
        ax[i].set_title(str(isf_name.sel(Nisf=kisf).values))
        #ax[i].set_xlim(0,60)
        #ax[i].axvline(x=30, c='k', linestyle='--')

        i = i+1

    #f.legend()
    #f.subplots_adjust(bottom=0.05, wspace=0.1)

    f.tight_layout()
    sns.despine()

    return f

SMB

model

In [ ]:
inputpath_atmo_Chris = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out/'
inputpath_atmo_Nico = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/SMB_EMULATED/'
scenario = 'historical'

melt_atmo_list = []
for mod in ['CESM2','CNRM-CM6-1','IPSL-CM6A-LR','MPI-ESM1-2-HR','UKESM1-0-LL']:
    if os.path.exists(inputpath_atmo_Chris+mod+'_SMB_'+scenario+'-1980-2014.nc'):
        melt_atmo = xr.open_dataset(inputpath_atmo_Chris+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
        melt_atmo['time'] = melt_atmo['time'].dt.year
        melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))
    else:
        melt_atmo = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
        melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))
        
for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2-WACCM','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 
           'MRI-ESM2-0']:
    melt_atmo = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
    melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))
    
melt_atmo_all = xr.concat(melt_atmo_list, dim='model')

reanalysis

In [ ]:
scenario = 'historical'
SMB_ERA5 = xr.open_dataset(inputpath_atmo_Chris+'ERA5_SMB_historical-1980-2014.nc').sel(Nisf=rignot_isf)
SMB_ERA5['time'] = SMB_ERA5['time'].dt.year

In [ ]:
f = figure_check_hist(sorted_isf_rignot,file_isf['isf_name'],davison_file['SMB_obs'],davison_file['SMB_unc'],
                      melt_atmo_all,obs2=SMB_ERA5['SMBorg'].mean('time'),obs2_unc=SMB_ERA5['SMBorg'].mean('time')*0, obs2_bool=True)

In [ ]:
f.savefig(plot_path + 'SMB_historical_check_with_SMBorigChris.pdf')

BASAL MELT

model

In [ ]:
param_classic_list = ['linear_local',
              'quadratic_local','quadratic_local_locslope',
              'lazero19',
              'boxes_4_pismyes_picopno']

param_NN_list = ['NNsmall']

In [ ]:
## Melt outputpath
Gt_allmod_list = []
box1_allmod_list = []

scenario = 'historical'
for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
           'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']:
#for mod in ['CESM2','CNRM-CM6-1','IPSL-CM6A-LR','MPI-ESM1-2-HR','UKESM1-0-LL']:
    
    outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'

    melt1D_list = []
    for mparam in param_classic_list:
        melt1D_scenario = xr.open_dataset(outputpath_melt + 'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS_anoISMIP.nc')
        melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_classic = xr.concat(melt1D_list, dim='param')       
    Gt_classic = melt1D_classic['melt_1D_Gt_per_y']

    melt1D_list = []
    for mparam in param_NN_list:
        melt1D_scenario = xr.open_dataset(outputpath_melt + 'evalmetrics_1D_'+mparam+'_'+scenario+'_anoISMIP.nc')
        melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_NN = xr.concat(melt1D_list, dim='param')   
    Gt_NN = melt1D_NN['predicted_melt'].sel(metrics='Gt')

    Gt_all = xr.concat([Gt_classic, Gt_NN], dim='param')
    
    Gt_allmod_list.append(Gt_all.assign_coords({'model': mod}))

Gt_allmod = xr.concat(Gt_allmod_list, dim='model')
    

obs estimate from Paolo

In [ ]:
melt_Paolo = xr.open_dataset(inputpath_raw + 'basal_melt_Gt_Paolo23.nc')

In [ ]:
melt_Paolo

In [ ]:
f = figure_check_hist(sorted_isf_rignot,file_isf['isf_name'],davison_file['basal_melt_obs'],davison_file['basal_melt_unc'],
                      Gt_allmod,obs2=melt_Paolo['melt_Gt'].mean('time'),obs2_unc=melt_Paolo['melt_Gt_unc'].mean('time'), obs2_bool=True)

In [ ]:
f.savefig(plot_path + 'BMB_historical_check_anoISMIP.pdf')

In [ ]:
f = figure_check_hist(sorted_isf_rignot,file_isf['isf_name'],davison_file['basal_melt_obs'],davison_file['basal_melt_unc'],
                      Gt_allmod.isel(param=0).drop('param')*0,obs2=melt_Paolo['melt_Gt'].mean('time'),obs2_unc=melt_Paolo['melt_Gt_unc'].mean('time'), obs2_bool=True)

In [ ]:
f.savefig(plot_path + 'compare_ref_BMB_davison_paolo.pdf')

GL flux

In [ ]:
GL_flux_ABUMIP = xr.open_dataset(outputpath_GL + 'all_GL_fluxes.nc')

In [ ]:
f = figure_check_hist(sorted_isf_rignot,file_isf['isf_name'],davison_file['discharge_obs'],davison_file['discharge_unc'],
                      Gt_allmod.isel(param=0).drop('param')*0,obs2=GL_flux_ABUMIP['flux_Gt_ref'],obs2_unc=GL_flux_ABUMIP['flux_Gt_ref']*0, obs2_bool=True)

In [ ]:
f.savefig(plot_path + 'compare_GL_flux_davison_cyrille.pdf')

BMB + CALVING - SMB

In [ ]:
davison_file 

In [ ]:
mass_balance_davison = davison_file['basal_melt_obs'] - davison_file['SMB_obs'] + davison_file['calving_obs']
mass_balance_davison_unc = np.sqrt(davison_file['basal_melt_unc']**2 + davison_file['SMB_unc']**2 + davison_file['calving_obs']**2)

In [ ]:
mass_balance_simu = Gt_allmod - melt_atmo_all + davison_file['calving_obs']

In [ ]:
f = figure_check_hist(sorted_isf_rignot,file_isf['isf_name'],mass_balance_davison,mass_balance_davison_unc,
                      mass_balance_simu,obs2=GL_flux_ABUMIP['flux_Gt_ref'],obs2_unc=GL_flux_ABUMIP['flux_Gt_ref']*0, obs2_bool=False)

In [ ]:
f.savefig(plot_path + 'alltogether_historical_check.pdf')